In [1]:
import xgboost as xgb
import pandas as pd
import databento_sqlmod1 as dbs
#import databento_sql as dbs
import fmp_keymetrics
import combiner
import numpy as np
import ta  # ta-lib or ta for calculating technical indicators
from datetime import datetime
from tqdm import tqdm

In [2]:
ticker_list = ['AAPL', 'MSFT', 'NVDA',  # Technology
    'AMZN', 'TSLA', 'HD',  # Consumer Discretionary
    'UNH', 'JNJ', 'LLY',  # Healthcare
    'JPM', 'BAC', 'WFC',  # Financials
    'XOM', 'CVX', 'COP',  # Energy
    'PG', 'KO', 'PEP',  # Consumer Staples
    'BA', 'CAT', 'UPS',  # Industrials
    'LIN', 'APD', 'SHW',  # Materials
    'PLD', 'AMT', 'CCI',  # Real Estate
    'NEE', 'DUK', 'SO',  # Utilities
]

In [3]:
# Databento download
for ticker in ticker_list:
    dbs.download_and_append_data(ticker, '2019-01-01', '2023-12-31', frequency='daily')

#FMP Download
start_date = '2019-01-01'
end_date = '2024-09-31'

fmp = fmp_keymetrics.funda_ETL()
keymetrics = fmp.download_funda_data(ticker_list, start=start_date, end=end_date)
keymetrics

# Merge Databento and FMP data
comb = combiner.combine_pvol_funda()
#tickers = ['AAPL','AMZN','GOOGL','MSFT']
merged = comb.merge_pvol_funda(ticker_list)

# Initialize combiner instance
comb = combiner.combine_pvol_funda()

def add_technical_indicators(df):
    # Add SMA, EMA
    df['sma_20'] = ta.trend.sma_indicator(df['close'], window=20)
    df['sma_50'] = ta.trend.sma_indicator(df['close'], window=50)
    df['ema_20'] = ta.trend.ema_indicator(df['close'], window=20)
    df['ema_50'] = ta.trend.ema_indicator(df['close'], window=50)
    df['ema_100'] = ta.trend.ema_indicator(df['close'], window=100)
    df['ema_200'] = ta.trend.ema_indicator(df['close'], window=200)
    
    # Add RSI, MACD
    df['rsi_14'] = ta.momentum.rsi(df['close'], window=14)
    df['macd'] = ta.trend.macd(df['close'])
    df['macd_signal'] = ta.trend.macd_signal(df['close'])

    # Add Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df['close'], window=20)
    df['bb_upper'] = bollinger.bollinger_hband()
    df['bb_lower'] = bollinger.bollinger_lband()
    
    # Add Average True Range (ATR)
    df['atr_14'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
    
    # Add On-Balance Volume (OBV)
    df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    
    # Add Stochastic Oscillator (K, D)
    stoch = ta.momentum.stoch(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    df['stoch_k'] = stoch
    df['stoch_d'] = ta.momentum.stoch_signal(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    
    # Remove momentum since it was causing an error
    # Add Williams %R
    df['williams_r'] = ta.momentum.williams_r(df['high'], df['low'], df['close'], lbp=14)
    
    # Add Chaikin Money Flow (CMF)
    df['cmf'] = ta.volume.chaikin_money_flow(df['high'], df['low'], df['close'], df['volume'], window=20)
    
    return df

def prepdata(ticker, start_date, end_date):
    # Retrieve data for the ticker and date range
    data = comb.get_comb_data_from_postgresql(ticker, start_date, end_date)

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Sort the data by date
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Add technical indicators
    df = add_technical_indicators(df)

    # Calculate log returns as the target variable
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))

    # Drop rows with NaN values in log returns and technical indicators
    df = df.dropna()

    # Define the fundamental columns and technical indicators to use as features
    fundamental_columns = [
        'revenuePerShare', 'netIncomePerShare', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 
        'cashPerShare', 'bookValuePerShare', 'tangibleBookValuePerShare', 'shareholdersEquityPerShare', 
        'interestDebtPerShare', 'marketCap', 'enterpriseValue', 'peRatio', 'priceToSalesRatio', 
        'pocfratio', 'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales', 'enterpriseValueOverEBITDA', 
        'evToOperatingCashFlow', 'evToFreeCashFlow', 'earningsYield', 'freeCashFlowYield', 
        'debtToEquity', 'debtToAssets', 'netDebtToEBITDA', 'currentRatio', 'interestCoverage', 
        'incomeQuality', 'dividendYield', 'payoutRatio', 'salesGeneralAndAdministrativeToRevenue', 
        'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets', 'capexToOperatingCashFlow', 
        'capexToRevenue', 'capexToDepreciation', 'stockBasedCompensationToRevenue', 'grahamNumber', 
        'roic', 'returnOnTangibleAssets', 'grahamNetNet', 'workingCapital', 'tangibleAssetValue', 
        'netCurrentAssetValue', 'investedCapital', 'averageReceivables', 'averagePayables', 
        'averageInventory', 'daysSalesOutstanding', 'daysPayablesOutstanding', 
        'daysOfInventoryOnHand', 'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover', 
        'roe', 'capexPerShare'
    ]
    
    # Technical columns added
    technical_columns = [
        'sma_20', 'sma_50', 'ema_20', 'rsi_14', 'macd', 'macd_signal', 'bb_upper', 'bb_lower', 
        'atr_14', 'obv', 'stoch_k', 'stoch_d', 'williams_r', 'cmf', 'ema_50', 'ema_100', 'ema_200'
    ]

    # Fill missing values in the fundamental columns using forward fill, followed by zeros if necessary
    df[fundamental_columns + technical_columns] = df[fundamental_columns + technical_columns].ffill().fillna(0)

    # Define features (X) and target (y)
    X = df[fundamental_columns + technical_columns]
    y = df['log_return']

    # Convert features into an XGBoost DMatrix
    dmat = xgb.DMatrix(X, label=y, missing=np.nan)

    return dmat, y, X

Ticker AAPL already up-to-date
data/equity/usa/daily/aapl.csv has been successfully zipped into data/equity/usa/daily/aapl.zip.
Data for AAPL fetched from postgres, converted to LEAN format.
Ticker MSFT already up-to-date
data/equity/usa/daily/msft.csv has been successfully zipped into data/equity/usa/daily/msft.zip.
Data for MSFT fetched from postgres, converted to LEAN format.
Ticker NVDA already up-to-date
data/equity/usa/daily/nvda.csv has been successfully zipped into data/equity/usa/daily/nvda.zip.
Data for NVDA fetched from postgres, converted to LEAN format.
Ticker AMZN already up-to-date
data/equity/usa/daily/amzn.csv has been successfully zipped into data/equity/usa/daily/amzn.zip.
Data for AMZN fetched from postgres, converted to LEAN format.
Ticker TSLA already up-to-date
data/equity/usa/daily/tsla.csv has been successfully zipped into data/equity/usa/daily/tsla.zip.
Data for TSLA fetched from postgres, converted to LEAN format.
Ticker HD already up-to-date
data/equity/usa/

In [4]:
# Example of how to use the updated function
#ticker_list = ['AAPL']
for ticker in tqdm(ticker_list, desc="Processing tickers", unit="ticker"):
    start_date = '2019-01-01'
    end_date = '2022-12-31'

    # Prepare the training data using log returns
    dtrain, y_train, X_train = prepdata(ticker, start_date, end_date)

    ''' Setting Test Data '''
    start_date = '2023-01-01'
    end_date = '2023-12-31'

    # Prepare the test data
    dtest, y_test, X_test = prepdata(ticker, start_date, end_date)

Processing tickers: 100%|██████████| 30/30 [00:11<00:00,  2.64ticker/s]


# Backtesting Framework

In [5]:
# Helper methods
def log_trade(trade_log, ticker, date, position_type, trade_type, entry_price, exit_price=None, signal=None):
    """
    Log trade details into the trade_log DataFrame.
    """
    return_on_trade = None
    if exit_price is not None:
        return_on_trade = (exit_price - entry_price) / entry_price

    trade_log = trade_log.append({
        'ticker': ticker,
        'date': date,
        'position_type': position_type,  # 'long' or 'short'
        'trade_type': trade_type,  # 'buy' or 'sell'
        'entry_price': entry_price,
        'exit_price': exit_price,
        'return': return_on_trade,
        'signal': signal
    }, ignore_index=True)
    
    return trade_log

def calculate_portfolio_metrics(trade_log_df):
    # Calculate daily returns from the trade log
    trade_log_df['daily_return'] = trade_log_df['exit_price'] / trade_log_df['entry_price'] - 1

    # Check if there are valid returns before calculating metrics
    if trade_log_df['daily_return'].empty or trade_log_df['daily_return'].isna().all():
        return {'Sharpe Ratio': float('nan'), 'Sortino Ratio': float('nan'), 'Max Drawdown': float('nan')}

    # Calculate metrics if valid data exists
    sharpe_ratio = (trade_log_df['daily_return'].mean() / trade_log_df['daily_return'].std()) * np.sqrt(252)
    
    # Sortino ratio: calculate downside deviation
    downside_risk = trade_log_df[trade_log_df['daily_return'] < 0]['daily_return'].std()
    sortino_ratio = (trade_log_df['daily_return'].mean() / downside_risk) * np.sqrt(252) if downside_risk > 0 else float('nan')

    # Max drawdown calculation
    cumulative_return = (1 + trade_log_df['daily_return']).cumprod()
    drawdown = cumulative_return / cumulative_return.cummax() - 1
    max_drawdown = drawdown.min()

    return {
        'Sharpe Ratio': sharpe_ratio if trade_log_df['daily_return'].std() > 0 else float('nan'),
        'Sortino Ratio': sortino_ratio,
        'Max Drawdown': max_drawdown
    }


In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import joblib
from datetime import datetime

# Placeholder for ticker data (train and test)
ticker_list = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'TSLA', 'HD', 
               'UNH', 'JNJ', 'LLY', 'JPM', 'BAC', 'WFC', 
               'XOM', 'CVX', 'COP', 'PG', 'KO', 'PEP',
               'BA', 'CAT', 'UPS', 'LIN', 'APD', 'SHW', 
               'PLD', 'AMT', 'CCI', 'NEE', 'DUK', 'SO']

# Load the trained model
best_model = joblib.load('xgb_best_model.pkl')

# Initialize variables for backtesting
initial_cash = 100000  # Initial capital
cash = initial_cash
portfolio = {ticker: 0 for ticker in ticker_list}  # Track positions in each stock
stop_loss_days = {ticker: 0 for ticker in ticker_list}  # Days to wait after stop-loss is hit

# Initialize an empty trade log DataFrame
trade_log = pd.DataFrame(columns=['ticker', 'date', 'position_type', 'trade_type', 'entry_price', 'exit_price', 'return', 'signal'])

# Load your price data for 2022 and 2023
def get_data(ticker, start_date, end_date):
    """
    Fetch data from PostgreSQL; if missing, download from Databento.
    """
    # Check if data already exists in PostgreSQL
    df = dbs.get_data_from_postgresql(ticker, start_date, end_date)

    if df is None or df.empty:
        print(f"Data for {ticker} not found in PostgreSQL. Fetching from Databento.")
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        df = dbs.get_data_from_databento(ticker, start_date, end_date)

        dbs.upload_to_postgresql(df, ticker)

    df['date'] = pd.to_datetime(df['ts_event'])
    df = df.sort_values(by='date')

    return df

# Backtesting loop for 2023
for ticker in ticker_list:
    train_data = get_data(ticker, '2019-01-01', '2022-12-31')
    test_data = get_data(ticker, '2023-01-01', '2023-12-31')

    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')
    dtest, y_test, X_test = prepdata(ticker, '2023-01-01', '2023-12-31')

    y_pred = best_model.predict(X_test)

    for i in range(len(y_pred) - 1):
        if stop_loss_days[ticker] > 0:
            stop_loss_days[ticker] -= 1
            continue
        
        if i + 1 >= len(test_data):
            break

        next_open = test_data['open'].iloc[i + 1]
        predicted_change = y_pred[i]
        signal = 'long' if predicted_change > 0 else 'short'
        
        # Buy signal
        if predicted_change > 0.01:
            num_shares = cash // next_open
            portfolio[ticker] += num_shares
            cash -= num_shares * next_open

            # Log buy trade
            trade_log_entry = pd.DataFrame({
                'ticker': [ticker],
                'date': [test_data['date'].iloc[i + 1]],
                'position_type': ['long'],
                'trade_type': ['buy'],
                'entry_price': [next_open],
                'exit_price': [np.nan],
                'return': [np.nan],
                'signal': [signal]
            })

            trade_log = pd.concat([trade_log, trade_log_entry], ignore_index=True)

        # Sell signal
        elif predicted_change < -0.01 and portfolio[ticker] > 0:
            exit_price = next_open
            entry_price = trade_log[trade_log['ticker'] == ticker]['entry_price'].iloc[-1]  # Get the last buy price
            trade_return = (exit_price - entry_price) / entry_price

            # Log sell trade
            trade_log_exit = pd.DataFrame({
                'ticker': [ticker],
                'date': [test_data['date'].iloc[i + 1]],
                'position_type': ['long'],
                'trade_type': ['sell'],
                'entry_price': [entry_price],
                'exit_price': [exit_price],
                'return': [trade_return],
                'signal': [signal]
            })

            trade_log = pd.concat([trade_log, trade_log_exit], ignore_index=True)

            # Reset position after selling
            cash += portfolio[ticker] * exit_price
            portfolio[ticker] = 0

        # Stop-loss implementation
        if (next_open < 0.9 * test_data['open'].iloc[i]) and portfolio[ticker] > 0:
            cash += portfolio[ticker] * next_open
            portfolio[ticker] = 0
            stop_loss_days[ticker] = 5

# Calculate final portfolio value
final_portfolio_value = cash + sum([portfolio[ticker] * test_data['close'].iloc[-1] for ticker in ticker_list])

print(f"Final Portfolio Value: {final_portfolio_value}")

# Calculate portfolio performance metrics
portfolio_metrics = calculate_portfolio_metrics(trade_log)
print(portfolio_metrics)


C:\Users\MP\AppData\Local\Temp\ipykernel_32584\2583515102.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_log = pd.concat([trade_log, trade_log_entry], ignore_index=True)


Final Portfolio Value: 41244.710000000036
{'Sharpe Ratio': 7.811566692314103, 'Sortino Ratio': 25.243391447924836, 'Max Drawdown': -0.10260950528493795}


In [21]:
trade_log.to_csv('trade_log.csv', index=False)
# Need to add logic to close all positions at the end of the backtest
# for each trade, note if stopped out as well
# add a column for the trade duration
# add a column for rolling portfolio value

# Model Training

In [11]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor
from tqdm import tqdm  # Import tqdm for progress bar

# Combine training data for multiple tickers
tickers = ticker_list #tickerlist defined at the top of file
X_combined = []
y_combined = []

# Loop through tickers with progress bar
for ticker in tqdm(tickers, desc="Processing tickers"):
    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')
    X_combined.append(X_train)
    y_combined.append(y_train)

X_combined = pd.concat(X_combined)
y_combined = np.concatenate(y_combined)

# Initialize model and parameter grid
#xgb_regressor = XGBRegressor()
xgb_regressor = XGBRegressor(tree_method="hist", device="cuda") #GPu accelerated

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0.1, 1, 10],
    'lambda': [0.1, 1, 10],
    'gamma': [0, 0.1, 0.5],
    'min_child_weight': [1, 5, 10],
}

# Use TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform GridSearchCV with cross-validation
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=tscv, scoring='r2', verbose=1)
grid_search.fit(X_combined, y_combined)

# Print the best parameters and R² score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best R² Score: {grid_search.best_score_}")


Processing tickers: 100%|██████████| 30/30 [00:11<00:00,  2.64it/s]
c:\Users\MP\anaconda3\envs\algo\lib\site-packages\xgboost\core.py:160: UserWarning: [11:44:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


Fitting 5 folds for each of 26244 candidates, totalling 131220 fits


c:\Users\MP\anaconda3\envs\algo\lib\site-packages\xgboost\core.py:160: UserWarning: [11:44:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
c:\Users\MP\anaconda3\envs\algo\lib\site-packages\xgboost\core.py:160: UserWarning: [11:44:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
c:\Users\MP\anaconda3\envs\algo\lib\site-packages\xgboost\core.py:160: UserWarning: [11:44:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
c:\Users\MP\anaconda3\envs\algo\lib\s

KeyboardInterrupt: 

In [8]:
import joblib

# Save the best model
best_model = grid_search.best_estimator_
joblib.dump(best_model, 'xgb_best_model.pkl')


['xgb_best_model.pkl']

In [ ]:
# Load the saved model
best_model = joblib.load('xgb_best_model.pkl')


# Temp methods for troubleshooting

In [7]:
# Backtester get_data function
from datetime import datetime
# Load your price data for 2022 and 2023
def get_data(ticker, start_date, end_date):
    """
    Fetch data from PostgreSQL; if missing, download from Databento.
    """
    # Check if data already exists in PostgreSQL
    df = dbs.get_data_from_postgresql(ticker, start_date, end_date)  # Assuming this method exists in databento_sql.py

    if df is None or df.empty:
        # If data is missing, download from Databento
        print(f"Data for {ticker} not found in PostgreSQL. Fetching from Databento.")
        # Convert start_date and end_date to datetime objects
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        df = dbs.get_data_from_databento(ticker, start_date, end_date)  # Download from Databento and return DataFrame

        # Upload the data to PostgreSQL for future use
        dbs.upload_to_postgresql(df, ticker)

    # Ensure the 'date' column is datetime and sorted by date
    df['date'] = pd.to_datetime(df['ts_event'])
    df = df.sort_values(by='date')

    return df

train_data = get_data('AAPL', '2019-01-01', '2022-12-31')

In [8]:
import os
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd

def get_data_from_postgresql(ticker, start_date=None, end_date=None, schema='databento_ohlcv'):
    """
    Retrieves data for a given ticker from PostgreSQL database, optionally within a date range.
    """
    # Fetch credentials from environment variables
    pguser = os.getenv('pguser')
    pgpass = os.getenv('pgpass')
    pghost = os.getenv('pghost')

    # Database connection URL using environment variables
    db_url = f'postgresql://{pguser}:{pgpass}@{pghost}/FinancialData'
    engine = create_engine(db_url)

    try:
        # Build the query
        query = f'SELECT * FROM "{schema}"."{ticker}"'
        if start_date is not None and end_date is not None:
            # Ensure start_date and end_date are strings

            # Convert date string to datetime object (if further handling is needed)
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
            end_date = datetime.strptime(end_date, '%Y-%m-%d')

            query += f" WHERE ts_event BETWEEN '{start_date.strftime('%Y-%m-%d')}' AND '{end_date.strftime('%Y-%m-%d')}'"

        df = pd.read_sql(query, con=engine)

        # Ensure ts_event is parsed as timezone-aware datetime
        df['ts_event'] = pd.to_datetime(df['ts_event'], utc=True)

        return df
    except Exception as e:
        print(f"Error retrieving data for {ticker} from PostgreSQL: {e}")
        return None
    finally:
        engine.dispose()
train_data = get_data_from_postgresql('AAPL', '2019-01-01', '2022-12-31')

In [9]:
train_data

,ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
0,2019-01-02 00:00:00+00:00,35,2,14,153.88,158.840,144.51,146.09,10526569,AAPL
1,2019-01-03 00:00:00+00:00,35,2,14,145.99,146.660,142.00,142.58,24831049,AAPL
2,2019-01-04 00:00:00+00:00,35,2,14,144.00,148.540,143.61,148.11,16654238,AAPL
3,2019-01-07 00:00:00+00:00,35,2,14,148.80,148.900,145.90,147.70,14586570,AAPL
4,2019-01-08 00:00:00+00:00,35,2,14,148.65,151.815,148.00,151.00,10027250,AAPL
...,...,...,...,...,...,...,...,...,...,...
1003,2022-12-23 00:00:00+00:00,35,2,29,131.98,133.390,129.64,131.75,12203671,AAPL
1004,2022-12-27 00:00:00+00:00,35,2,29,132.60,132.860,128.72,130.00,14730147,AAPL
1005,2022-12-28 00:00:00+00:00,35,2,29,130.19,131.020,125.87,126.26,17277335,AAPL
1006,2022-12-29 00:00:00+00:00,35,2,29,126.62,130.480,126.45,129.36,16470915,AAPL
